### Step 0: Preliminaries
Necessary package installations and imports.

In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.0.1+cu117


### Step 1: Create a simple graph
Create a simple graph with three nodes and two edges

In [35]:
import redis
from redis.commands.graph import Graph
from redis.commands.graph.node import Node
from redis.commands.graph.edge import Edge

connection = redis.Redis('localhost', 6379, 0)
graph = Graph(connection, 'pyg')

jane = Node(label='person', properties={'age':32})
graph.add_node(jane)
john = Node(label='person', properties={'age':38})
graph.add_node(john)
japan = Node(label='country', properties={'name': 'Japan'})
graph.add_node(japan)
edge1 = Edge(jane, 'visited', japan, properties={'purpose': 'pleasure'})
graph.add_edge(edge1)
edge2 = Edge(john, 'visited', japan, properties={'purpose': 'pleasure'})
graph.add_edge(edge2)
graph.commit()




In [36]:
from database import Database
db = Database('localhost',6379)
feature_store, graph_store = db.get_torch_geometric_remote_backend()

In [37]:
feature_store['person', 'age', 32]


AttributeError: 'RedisGraphFeatureStore' object has no attribute '_tensor_attr_cls'